In [1]:
import xarray as xr
import pandas as pd
from pathlib import Path
import ast
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import os
from shapely.ops import unary_union


In [2]:
# monthly averages 
sfe = xr.open_dataset('/scratch/users/ashdef/SFE_monthly/SFE_monthly_skipna.nc')

precip = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/pr_monthly.nc')

rmax = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/rmax_monthly.nc')

rmin = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/rmin_monthly.nc')

srad = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/srad_monthly.nc')

tmin = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/tmmn_monthly.nc')

tmax = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/tmmx_monthly.nc')

vpd = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/vpd_monthly.nc')

ws = xr.open_dataset('/scratch/users/ashdef/GridMET_monthly/vs_monthly.nc')

## ET and climate data to only include treated pixels

In [3]:
# treatment data
# overlapping pixels have already been merged if their treatments occurred <3 days apart
# only contains WUS treatments
twig_data = gpd.read_file('/oak/stanford/groups/konings/ashdef/TWIG/TWIG_3day_no_overlap/wus_trmts_merged_3day.shp')
twig_data

,treatment_,twig_categ,geometry
0,2002-09-01,Mechanical,"POLYGON Z ((-12393260.542 5637811.54 -100000, ..."
1,1990-08-01,Mechanical,"POLYGON Z ((-12866381.239 5748452.649 -100000,..."
2,2022-07-01,"Mechanical, Planned Ignition, Unplanned Ignition",MULTIPOLYGON Z (((-12681223.54 6141885.97 -100...
3,1993-07-01,Mechanical,"POLYGON Z ((-13024599.581 6240766.296 -100000,..."
4,1998-04-01,Planned Ignition,"POLYGON Z ((-11840763.606 5658727.627 -100000,..."
...,...,...,...
80816,2023-10-30,Mechanical,"POLYGON Z ((-12687623.84 5664566.575 -100000, ..."
80817,2025-10-30,Mechanical,"POLYGON Z ((-12686588.196 5660323.458 -100000,..."
80818,2010-10-20,Mechanical,"POLYGON Z ((-11829665.679 5012370.196 -100000,..."
80819,2011-10-19,Planned Ignition,"POLYGON Z ((-11813087.771 5067130.862 -100000,..."


### turn sfe pixels into polygons for spatial join

https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html

In [5]:
# modified chatgpt code
lat = sfe['lat'].values
lon = sfe['lon'].values

# resolution of pixels in terms of lat/lon
res_lat = abs(lat[1] - lat[0])
res_lon = abs(lon[1] - lon[0])

polygons = []
pixel_info = []

for i, lat_val in enumerate(lat):
    for j, lon_val in enumerate(lon):

        # turn the pixel into a polygon box
        # xmin/max and ymin/max are half of the resolution from the centroid  
        cell = box(
            lon_val - res_lon/2, lat_val - res_lat/2,
            lon_val + res_lon/2, lat_val + res_lat/2
        )
        polygons.append(cell)

        # save the inde of each pixel, and the lat/lon from sfe
        pixel_info.append({
            "pixel_id": (i,j),
            "lat": lat_val,
            "lon": lon_val
        })

sfe_gdf = gpd.GeoDataFrame(pixel_info, geometry=polygons, crs="EPSG:4326")


# reproject to match treatment polygons
sfe_gdf = sfe_gdf.to_crs(twig_data.crs)

sfe_gdf.to_file("/scratch/users/ashdef/SFE_monthly/geodataframes/handmade.shp")

In [4]:
sfe_gdf = gpd.read_file("/scratch/users/ashdef/SFE_monthly/geodataframes/handmade.shp")

In [5]:
# geometry column of the output belongs to the pixels
# index_right column of the output is the index of the intersecting polygon from twig
pixel_treatment = gpd.sjoin(sfe_gdf, twig_data, how="inner", predicate="intersects")
print(f"{len(pixel_treatment)} intersecting pixels")

114471 intersecting pixels


### ensure the dataframe with pixel and treatment info includes the treatment geometries

In [6]:
pixel_treatment = pixel_treatment.rename(columns={'treatment_':'treatment_date'})

In [7]:
pixel_treatment = pixel_treatment.rename(columns={'geometry':'pixel_geometry'})

In [8]:
twig_data_intersects = twig_data.loc[pixel_treatment['index_right']]

In [9]:
twig_data_intersects

,treatment_,twig_categ,geometry
7345,1985-08-01,Unknown,"POLYGON Z ((-12954559.683 6276174.041 -100000,..."
7345,1985-08-01,Unknown,"POLYGON Z ((-12954559.683 6276174.041 -100000,..."
7044,1996-09-01,Mechanical,"POLYGON Z ((-12950618.275 6276430.518 -100000,..."
6993,1996-09-01,Mechanical,"POLYGON Z ((-12950123.344 6277202.513 -100000,..."
12464,1992-09-01,Mechanical,"POLYGON Z ((-12948989.931 6279191.263 -100000,..."
...,...,...,...
51574,2014-01-17,Planned Ignition,"POLYGON Z ((-12164113.978 3676579.982 -100000,..."
52426,2020-02-16,Planned Ignition,MULTIPOLYGON Z (((-12164305.482 3676239.064 -1...
50673,2019-05-24,Mechanical,"POLYGON Z ((-10538200.377 3587911.819 -100000,..."
50285,2019-05-24,Mechanical,"POLYGON Z ((-10501010.788 3580202.916 -100000,..."


In [10]:
pixel_treatment['treatment_geometry'] = twig_data_intersects['geometry'].values

In [11]:
pixel_treatment

,pixel_id,lat,lon,pixel_geometry,index_right,treatment_date,twig_categ,treatment_geometry
12675,"(9, 201)",49.025000,-116.391667,"POLYGON ((-12954341.906 6275568.42, -12958980....",7345,1985-08-01,Unknown,"POLYGON Z ((-12954559.683 6276174.041 -100000,..."
12676,"(9, 202)",49.025000,-116.350000,"POLYGON ((-12949703.594 6275568.42, -12954341....",7345,1985-08-01,Unknown,"POLYGON Z ((-12954559.683 6276174.041 -100000,..."
12676,"(9, 202)",49.025000,-116.350000,"POLYGON ((-12949703.594 6275568.42, -12954341....",7044,1996-09-01,Mechanical,"POLYGON Z ((-12950618.275 6276430.518 -100000,..."
12676,"(9, 202)",49.025000,-116.350000,"POLYGON ((-12949703.594 6275568.42, -12954341....",6993,1996-09-01,Mechanical,"POLYGON Z ((-12950123.344 6277202.513 -100000,..."
12676,"(9, 202)",49.025000,-116.350000,"POLYGON ((-12949703.594 6275568.42, -12954341....",12464,1992-09-01,Mechanical,"POLYGON Z ((-12948989.931 6279191.263 -100000,..."
...,...,...,...,...,...,...,...,...
601896,"(434, 372)",31.316667,-109.266667,"POLYGON ((-12161190.534 3671228.625, -12165828...",51574,2014-01-17,Planned Ignition,"POLYGON Z ((-12164113.978 3676579.982 -100000,..."
601896,"(434, 372)",31.316667,-109.266667,"POLYGON ((-12161190.534 3671228.625, -12165828...",52426,2020-02-16,Planned Ignition,MULTIPOLYGON Z (((-12164305.482 3676239.064 -1...
624422,"(450, 722)",30.650000,-94.683333,"POLYGON ((-10537781.294 3584682.444, -10542419...",50673,2019-05-24,Mechanical,"POLYGON Z ((-10538200.377 3587911.819 -100000,..."
625816,"(451, 730)",30.608333,-94.350000,"POLYGON ((-10500674.797 3579293.244, -10505313...",50285,2019-05-24,Mechanical,"POLYGON Z ((-10501010.788 3580202.916 -100000,..."


### filter pixel / treatment info if they contain multiple treatments, and if their treatment date is 1985 (no valid pre-treatment data)
- If treatments within the pixel occurred in the same calendar year: keep the latest treatment date, concatenate treatment category, and merge treatment geometries


In [13]:
pixel_treatment['treatment_date'] = pd.to_datetime(pixel_treatment['treatment_date'])

# contains the earliest and latest treatment date for all pixels
# if pixels are only treated once, these dates are the same
same_pixel = pixel_treatment.groupby(pixel_treatment.index)['treatment_date'].agg(['min', 'max'])
same_pixel

,min,max
12675,1985-08-01,1985-08-01
12676,1985-08-01,1996-09-01
12677,1986-09-01,1996-09-01
12678,1985-08-01,1985-08-01
12679,1985-08-01,1985-08-01
...,...,...
601869,2023-03-30,2023-03-30
601896,2014-01-17,2020-02-16
624422,2019-05-24,2019-05-24
625816,2019-05-24,2019-05-24


In [14]:
# a list of the indices where the earliest and latest treatment dates have the same year
# want to keep these
indices_to_keep = same_pixel[(same_pixel['max'].dt.year == same_pixel['min'].dt.year)].index
indices_to_keep

Index([ 12675,  12678,  12679,  12681,  13948,  13949,  13973,  13974,  13975,
        13976,
       ...
       600487, 600492, 601862, 601863, 601866, 601867, 601869, 624422, 625816,
       627202],
      dtype='int64', length=14233)

In [15]:
# filter original treatment df to only include the indices to keep
pixel_treatment = pixel_treatment.loc[indices_to_keep]



In [16]:
# concatenate treatment category
# lambda: apply function to each element
twig_cat_concat = (pixel_treatment.groupby(pixel_treatment.index)['twig_categ'].apply(lambda s: ' '.join(s.astype(str))))
twig_cat_concat

12675              Unknown
12678           Mechanical
12679           Mechanical
12681           Mechanical
13948           Mechanical
                ...       
601867    Planned Ignition
601869          Mechanical
624422          Mechanical
625816          Mechanical
627202          Mechanical
Name: twig_categ, Length: 14233, dtype: object

In [ ]:
# modified code from chat gpt
# merge treatment polygons
trtmt_polygons_concat = (
    pixel_treatment.groupby(pixel_treatment.index)['treatment_geometry'].apply(lambda s: unary_union(s))
)

In [ ]:
# take latest treatment date
latest_trmt_dates = (pixel_treatment.groupby(pixel_treatment.index)['treatment_date'].apply(lambda t: max(t)))

In [ ]:
# group by pixels, choosing the last row for that pixel although it doesn't really matter bc merged necessary info
# add concatenated treatment categories to the dataframe
pixel_treatment = (pixel_treatment.groupby(pixel_treatment.index).last().assign(treatment_date = latest_trmt_dates, twig_categ = twig_cat_concat, treatment_geometry = trtmt_polygons_concat))

In [ ]:
pixel_treatment

In [ ]:
# get rid of pixels treated in 1985, and beyond outside of sfe range
start_date = pd.to_datetime('1986-01-01')
end_date = pd.to_datetime('2023-12-31')

# filter to avoid future treatments and past treatments that technically shouldnt be there
pixel_treatment = pixel_treatment[(pixel_treatment['treatment_date'] >= start_date) & (pixel_treatment['treatment_date'] <= end_date)]


In [101]:
pixel_treatment.to_csv('pixel_treatment_clean.csv')

### filter sfe and climate xarrays to only include pixels remaining after filtering pixel_treatment df

In [3]:
pixel_treatment = pd.read_csv('pixel_treatment_clean.csv')

In [4]:
pixel_treatment = pixel_treatment.rename(columns={'Unnamed: 0' : 'sfe_gdf_index'})

In [5]:
pixel_treatment

,sfe_gdf_index,pixel_id,lat,lon,pixel_geometry,index_right,treatment_date,twig_categ,treatment_geometry
0,12681,"(9, 207)",49.025000,-116.141667,POLYGON ((-12926512.033446606 6275568.41973754...,2952,2001-07-25,Mechanical,POLYGON Z ((-12930454.5056 6275684.098099999 -...
1,13948,"(10, 88)",48.983333,-121.100000,POLYGON ((-13478471.175296586 6268500.82233061...,53026,2022-09-28,Mechanical,MULTIPOLYGON Z (((-13478494.3715 6271525.47070...
2,13949,"(10, 89)",48.983333,-121.058333,POLYGON ((-13473832.863180202 6268500.82233061...,53026,2022-09-28,Mechanical,MULTIPOLYGON Z (((-13478494.3715 6271525.47070...
3,13973,"(10, 113)",48.983333,-120.058333,POLYGON ((-13362513.372386927 6268500.82233061...,78185,2023-10-09,Unplanned Ignition,POLYGON Z ((-13363361.2061 6274779.129100002 -...
4,13974,"(10, 114)",48.983333,-120.016667,POLYGON ((-13357875.060270542 6268500.82233061...,78185,2023-10-09,Unplanned Ignition,POLYGON Z ((-13363361.2061 6274779.129100002 -...
...,...,...,...,...,...,...,...,...,...
12555,601867,"(434, 343)",31.316667,-110.475000,POLYGON ((-12295701.585618054 3671228.62456537...,24208,2023-05-11,Planned Ignition,POLYGON Z ((-12301319.7962 3676068.6059999987 ...
12556,601869,"(434, 345)",31.316667,-110.391667,POLYGON ((-12286424.961385284 3671228.62456537...,54017,2023-03-30,Mechanical,MULTIPOLYGON Z (((-12288803.2709 3677781.14090...
12557,624422,"(450, 722)",30.650000,-94.683333,POLYGON ((-10537781.29350761 3584682.444206471...,50673,2019-05-24,Mechanical,POLYGON Z ((-10538200.3766 3587911.8193999976 ...
12558,625816,"(451, 730)",30.608333,-94.350000,POLYGON ((-10500674.796576519 3579293.24373258...,50285,2019-05-24,Mechanical,POLYGON Z ((-10501010.788 3580202.9158999994 -...


In [6]:
sfe

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 468, lat: 585, lon: 1386)
Coordinates:
  * lat      (lat) float64 5kB 49.4 49.36 49.32 49.28 ... 25.15 25.11 25.07
  * lon      (lon) float64 11kB -124.8 -124.7 -124.7 ... -67.14 -67.1 -67.06
  * time     (time) datetime64[ns] 4kB 1985-01-01 1985-02-01 ... 2023-12-01
Data variables:
    ET       (time, lat, lon) float64 3GB ...

In [7]:
precip

<xarray.Dataset> Size: 3GB
Dimensions:               (time: 468, lat: 585, lon: 1386, crs: 1)
Coordinates:
  * lon                   (lon) float64 11kB -124.8 -124.7 ... -67.1 -67.06
  * lat                   (lat) float64 5kB 49.4 49.36 49.32 ... 25.11 25.07
  * crs                   (crs) uint16 2B 3
  * time                  (time) datetime64[ns] 4kB 1985-01-01 ... 2023-12-01
Data variables:
    precipitation_amount  (time, lat, lon) float64 3GB ...
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    date:                       03 July 2019
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [8]:
# get the lat/lon values of filtered pixels containing treatment
lat_vals = pixel_treatment['lat'].values
lon_vals = pixel_treatment['lon'].values

lat_mask = np.isin(sfe['lat'], lat_vals)
lon_mask = np.isin(sfe['lon'], lon_vals)

# select only the pixels that intersect treatment
sfe_filtered = sfe.sel(lat=lat_mask, lon=lon_mask)
print(sfe_filtered)


<xarray.Dataset> Size: 472MB
Dimensions:  (time: 468, lat: 350, lon: 360)
Coordinates:
  * lat      (lat) float64 3kB 49.03 48.94 48.9 48.82 ... 31.65 31.52 31.4 30.65
  * lon      (lon) float64 3kB -124.4 -124.4 -124.3 ... -90.64 -83.72 -83.68
  * time     (time) datetime64[ns] 4kB 1985-01-01 1985-02-01 ... 2023-12-01
Data variables:
    ET       (time, lat, lon) float64 472MB ...


In [9]:
sfe_filtered

<xarray.Dataset> Size: 472MB
Dimensions:  (time: 468, lat: 350, lon: 360)
Coordinates:
  * lat      (lat) float64 3kB 49.03 48.94 48.9 48.82 ... 31.65 31.52 31.4 30.65
  * lon      (lon) float64 3kB -124.4 -124.4 -124.3 ... -90.64 -83.72 -83.68
  * time     (time) datetime64[ns] 4kB 1985-01-01 1985-02-01 ... 2023-12-01
Data variables:
    ET       (time, lat, lon) float64 472MB ...

In [10]:
# filter each climate dataset
climate_datasets = {
    'precip': precip,
    'rmax': rmax,
    'rmin': rmin,
    'srad': srad,
    'tmin': tmin,
    'tmax': tmax,
    'vpd': vpd,
    'ws': ws
}

climate_filtered = {}

# help from chatgpt to optimize this loop
for name, ds in climate_datasets.items():
    ds_filtered = ds.sel(lat=lat_mask, lon=lon_mask)
    climate_filtered[name] = ds_filtered


In [11]:
climate_filtered['precip']

<xarray.Dataset> Size: 472MB
Dimensions:               (time: 468, lat: 350, lon: 360, crs: 1)
Coordinates:
  * lon                   (lon) float64 3kB -124.4 -124.4 ... -83.72 -83.68
  * lat                   (lat) float64 3kB 49.03 48.94 48.9 ... 31.4 30.65
  * crs                   (crs) uint16 2B 3
  * time                  (time) datetime64[ns] 4kB 1985-01-01 ... 2023-12-01
Data variables:
    precipitation_amount  (time, lat, lon) float64 472MB ...
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    date:                       03 July 2019
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [12]:
sfe_filtered.to_netcdf("/scratch/users/ashdef/filtered_sfe_gridmet/sfe_filtered.nc")

In [12]:
# help from chatgpt to optimize this loop
for var_name, da in climate_filtered.items():
    da.to_netcdf(f"/scratch/users/ashdef/filtered_sfe_gridmet/{var_name}_filtered.nc")
    print(f"Saved {var_name}")

Saved precip
Saved rmax
Saved rmin
Saved srad
Saved tmin
Saved tmax
Saved vpd
Saved ws
